In [0]:



date="2020-01-08"
df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/device_code=android-all/".format(date))
print df.filter("app_id==20600009602008 ").show(20)


In [0]:



date="2020-01-01"
df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est.v1/fact/granularity=daily/date={}/device_code=ios-all/".format(date))
print df.filter("app_id=20600010387037").show(20)




In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)


In [0]:


import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_0
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
""".format(date)

    db_df = query_df(daily_est_dsn, sql)
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/device_code=android-all/".format(date)).select("app_id").distinct().toPandas()

    u_list = unified_df.app_id.unique()
    d_list = db_df.app_id.unique()
    diff_list =  [x for x in  d_list if x not in u_list ]
     
    if len(diff_list) ==  0:
        print "{} PASS".format(date)
    else:
        print "{} FAIL".format(date)
        print "{} {} {}".format(len(u_list), len(d_list), diff_list[0:5])
        # print len(u_list)
        # print len(d_list)
        # # print [x for x in  u_list if x not in d_list ]
        # print [x for x in  d_list if x not in u_list ]
        # print '*' * 200
        


date_list = get_date_list("2019-11-01", "2020-02-21")
for date in date_list:
    # compare(date)
    try:
        compare(date)
    except Exception, e:
        print "{}: ERROR".format(date)
        print e.message
